In [7]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [8]:
#Carregar a tabela de buscas de produtos:
produtos_df = pd.read_excel(r'Base de Buscas\buscas.xlsx')
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [9]:
#Abrir o site do Google Shopping:
navegador.get("https://shopping.google.com.br/")

In [10]:
#Buscar o produto:
navegador.find_element(By.ID, 'REsRA').send_keys(produtos_df.loc[0, 'Nome'])
navegador.find_element(By.CLASS_NAME, 'uDgUL').click()

In [11]:
preco_minimo = float(produtos_df.loc[0, 'Preço mínimo'])
preco_maximo = float(produtos_df.loc[0, 'Preço máximo'])

In [12]:
lista_ofertas_google = []

#Pegar todos os produtos da página:
lista_produtos = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__content')

for produto in lista_produtos:
    #Obter os dados dos produtos:
    nome_produto = produto.find_element(By.TAG_NAME, 'h3').text.lower().replace(',','')
    preco_produto = produto.find_element(By.CLASS_NAME, 'kHxwFf').text.split(" ")
    preco_produto = preco_produto[1].replace('\nR$', '').replace('.', '').replace(',', '.')
    
    #Filtrar pelos dados da busca:
    lista_termos_produto = produtos_df.loc[0, 'Nome'].lower().split(" ")
    lista_termos_banidos = produtos_df.loc[0, 'Termos banidos'].lower().split(" ")

    termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)

    termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

    if termo_prod_correto and not termo_banido_encontrado:
        if preco_minimo <= float(preco_produto) <= preco_maximo:
            link_produto = produto.find_element(By.TAG_NAME, 'a').get_attribute('href')
            lista_ofertas_google.append((nome_produto, preco_produto, link_produto))

print(lista_ofertas_google)

[('vitrine: iphone 12 apple 64gb roxo tela 61 câmera traseira 12mp ios', '3286.05', 'https://www.google.com.br/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-roxo-tela-61-camera-traseira-12mp-ios-mp932337971/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwj7172no77-AhWDrZUCHfGZANkQ1SkItQwoAA&usg=AOvVaw1USPJHSB6JcAVhrFM4DrE5'), ('iphone 12 64gb - roxo - estou zerado', '3059.10', 'https://www.google.com.br/shopping/product/1409198851555036304?hl=pt-BR&psb=1&q=iphone+12+64gb&oq=iphone+12+64gb&gs_lcp=Cgtwcm9kdWN0cy1jYxAMUA1YKGBIaAFwAHgAgAEAiAEAkgEAmAEAoAEBsAEA&sclient=products-cc&prds=eto:14066938966366365827_0,pid:5070126177312031027,rsk:PROD_VAR_925660058154905600&sa=X&ved=0ahUKEwj7172no77-AhWDrZUCHfGZANkQ8gIIxQwoAA'), ('vitrine: iphone 12 apple 64gb preto tela 61 câmera traseira dupla 12mp ios', '3286.05', 'https://www.google.com.br/shopping/product/10731725505291757596?hl=pt-BR&psb=1&q=iphone+12+64gb&oq=iphone+12+64gb&gs_lcp=Cgtwcm9kdWN0cy1jYxAMUA1YKGBIaAFwAHgAgAEAiAEAkgEAmAEAoAEB